# [INDEX](#INDEX)
* [Time_Series](#Time_Series)
* [Country_Code](#Country_Code)
* [Day_Change](#Day_Change)
* [Death_Rate](#Death_Rate)
* [County_Profiles](#County_Profiles)
* [US_by_County](#US_by_County)


In [1]:
%matplotlib inline
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
from datetime import datetime
import re
import io
import requests
import sys

sns.set(style="darkgrid")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# [Time_Series](#INDEX)

# [US_by_County](#INDEX)

In [2]:
import io
import requests
# For confirmed Cases 
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
s=requests.get(url).content
US_county_time_series=pd.read_csv(url)
US_county_time_series =US_county_time_series.fillna(0)
US_county_time_series['FIPS'] = US_county_time_series['FIPS'].astype(int)
US_county_time_series['FIPS'] = US_county_time_series['FIPS'].apply(lambda x: '{0:0>5}'.format(x)).astype(str)
US_county_time_series.to_csv('./resources/US_county_time_series.csv')
last_day = US_county_time_series.columns[-1]
US_county_time_series.sort_values(by = last_day, ascending = False).head(10)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20
1863,84036061,US,USA,840,36061,New York,New York,US,40.767273,-73.971526,"New York City, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,4,11,11,12,19,25,55,95,154,269,269,463,814,1871,3954,5151,7530,9654,12305,14904,17856,21873,25573,29776,33768,37453,43119,47439,51809,57159,63306,67551,72181,76876,81803,87028,92384,98308,103208,106763,110465,118302,123146,127352,131334,134267,136591,139348,142227,145855,150473,155113,158258,160489,162338,164841,167478,169690,172354,174331,175651,176874,178351,180216,181783,183289
615,84017031,US,USA,840,17031,Cook,Illinois,US,41.841448,-87.816588,"Cook, Illinois, US",0,0,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,4,4,4,5,5,6,7,7,11,22,27,40,50,50,62,107,178,278,278,548,805,922,1194,1418,1418,2239,2613,3445,3727,4496,5152,5575,6111,7439,8034,8728,9509,10520,11415,12472,13417,14585,15474,16323,17306,18087,19391,20395,21272,22101,23181,24546,25811,27616,29058,30574,31953,33449,34880,36513,38668,40227,42324,43715,45223,46689,48341,50236,51674
1862,84036059,US,USA,840,36059,Nassau,New York,US,40.740665,-73.589419,"Nassau, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,5,17,19,28,41,51,68,79,109,140,278,372,754,1234,1900,2442,2869,3285,3914,4657,5537,6445,7344,8544,9554,10587,12024,13346,14398,15616,16610,18548,20140,21512,22584,23553,24358,25250,26715,27772,28539,29180,30013,30677,31079,31555,32124,32765,33798,34522,34865,35085,35505,35854,36161,36519,36780,36965,37152,37350,37593,37812,38028
1884,84036103,US,USA,840,36103,Suffolk,New York,US,40.883201,-72.801217,"Suffolk, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,6,20,28,33,41,74,97,116,239,459,662,1034,1458,1880,2260,2735,3385,4138,5023,5791,6713,7605,8746,10154,11370,13129,14185,15561,17008,18803,20320,20321,20934,21643,22691,23523,24812,25035,26143,26888,28127,28701,28854,29567,30606,31368,32059,32470,32724,33265,33664,34037,34478,34855,35077,35275,35543,35892,36223,36461
209,84006037,US,USA,840,06037,Los Angeles,California,US,34.308284,-118.228241,"Los Angeles, California, US",0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,11,13,14,14,14,20,27,32,40,53,53,94,144,190,231,292,292,407,536,662,812,1229,1465,1465,1829,2474,3019,3518,4045,4566,4605,5955,6377,6936,7559,7955,8443,8453,8894,9433,10047,10517,10854,11400,12021,12341,13823,15153,16447,17537,18545,19133,19567,20423,20996,22485,23220,24262,24894,25699,26225,27836,28665,29511,30319,31241
1892,84036119,US,USA,840,36119,Westchester,New York,US,41.162784,-73.757417,"Westchester, New York, US",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,18,19,57,83,98,98,121,148,158,172,178,220,380,538,798,1091,1387,1873,2894,3891,4691,5944,7187,7875,8519,9326,9967,10683,11567,12351,13081,13723,14294,14294,15887,17004,18077,18729,19313,19786,20191,20947,21828,22476,23179,23803,24306,24656,25276,25959,26633,27231,276

In [3]:
days = US_county_time_series.columns[11::]

In [4]:
US_county_confirmed_day_change=pd.DataFrame()
for i in range(len(days)-1):
    US_county_confirmed_day_change[days[i+1]]= US_county_time_series[days[i+1]] - US_county_time_series[days[i]]
US_county_confirmed_day_change_ = pd.concat((US_county_time_series.iloc[:,0:10], US_county_confirmed_day_change), axis = 1)
colnames = US_county_confirmed_day_change_.columns 
test = lambda x : 'county' if (x == 'Admin2') else x 
new_colnames = pd.Series(colnames).apply(test)
US_county_confirmed_day_change_.columns = new_colnames
US_county_confirmed_day_change_.to_csv('./resources/US_county_confirmed_day_change.csv')

In [5]:
# For deaths in US
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
s=requests.get(url).content
US_county_deaths_time_series=pd.read_csv(url)
US_county_deaths_time_series =US_county_deaths_time_series.fillna(0)
US_county_deaths_time_series['FIPS'] = US_county_deaths_time_series['FIPS'].astype(int)
US_county_deaths_time_series['FIPS'] = US_county_deaths_time_series['FIPS'].apply(lambda x: '{0:0>5}'.format(x)).astype(str)
US_county_deaths_time_series.to_csv('./resources/US_county_deaths_time_series.csv')


In [6]:
US_county_deaths_time_series.to_csv('./resources/US_county_deaths_time_series.csv')

In [7]:
US_county_deaths_day_change=pd.DataFrame()
for i in range(len(days)-1):
    US_county_deaths_day_change[days[i+1]]= US_county_deaths_time_series[days[i+1]] - US_county_deaths_time_series[days[i]]
US_county_deaths_day_change_ = pd.concat((US_county_deaths_time_series.iloc[:,0:4], US_county_deaths_day_change), axis = 1)
US_county_deaths_day_change_.to_csv('./resources/US_county_deaths_day_change.csv')
#US_county_deaths_day_change_

In [8]:
# ZTCA to place relationship data --https://www.census.gov/programs-surveys/geography/technical-documentation/records-layout/2010-zcta-record-layout.html#par_textimage_2
# It's a huge file. Already downloaded as 'ztca_rel_census.csv'
url = 'https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_county_rel_10.txt'
county_data = pd.read_csv(url)
#zcta = pd.read_csv('zcta_county_rel_census.csv')
#us_county_ztca_pop_area = zcta[['ZCTA5', 'PLACE','POPPT', 'AREALANDPT']] # population and land area for fip and ztca)
#county_data

In [9]:
US_County_Data = pd.read_excel('./resources/US_Country_Profile.xlsx')
#US_County_Data

In [10]:
# URLS for Datasets
covid_cases_data_url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
us_county_data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTa2WOoE0h9f4Ol-N_o4bJ-iAHJkwkAkcc0T1cSWpi0p4ebBR4CH-CbkcwjqUawrMXPoXtGBY1xlgEQ/pub?output=csv'

#Load dataframes
covid_cases_df = pd.read_csv(covid_cases_data_url)
us_county_df = pd.read_csv(us_county_data_url)

# Transformation for COVID19 cases
us_county_df = pd.merge(left = us_county_df, right = covid_cases_df.groupby('fips').sum().rename(columns={'fips':'fips','cases' : 'CONFIRMED_CASES', 'deaths' : 'DEATHS'}), left_on = 'FIPS', right_on = 'fips')
us_county_df = pd.merge(left = us_county_df, right = covid_cases_df.groupby('fips').count().rename(columns={'fips':'fips','cases' : 'DAYS_SINCE_FIRST_CASE'})['DAYS_SINCE_FIRST_CASE'], left_on = 'FIPS', right_on = 'fips')
us_county_df['COVID_CONFIRMED_CASES_10k'] = (us_county_df.CONFIRMED_CASES / us_county_df['POPESTIMATE2019']) * 10000.0
us_county_df['COVID_CONFIRMED_CASES_GROWTH_RATE'] = us_county_df.CONFIRMED_CASES / us_county_df.DAYS_SINCE_FIRST_CASE
us_county_df['COVID_CONFIRMED_CASES_GROWTH_RATE_10k'] = us_county_df.COVID_CONFIRMED_CASES_10k / us_county_df.DAYS_SINCE_FIRST_CASE


In [11]:
us_county_df.to_csv('./resources/us_county_df.csv')